# Salario de executivos

Nesse estudo de caso será considerado o salário de 220 executivos (145 homens e 75 mulheres), de diversas idades e tempo de experiencia no cargo.

As informações estão distribuidas da seguinte maneira:
- Sexo (1 para masculino e 2 para feminino)
- Anos de Experiencia 
- Posição na empresa (varia de 1 a 9, quanto maior o valor mais alta a posição)


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [3]:
df = pd.read_csv('salary.csv', names=['salario', 'sexo', 'posicao', 'exp'])
df.head()

,salario,sexo,posicao,exp
0,148,Masculino,7,16.7
1,165,Masculino,7,6.7
2,145,Masculino,5,14.8
3,139,Feminino,7,13.9
4,142,Feminino,6,6.4


O livro nos mostra na analise prévia que em graficos de dispersão que o crescimento do salário é acompanhado pelo tempo de experiencia e crescimento na posição, tanto para homens quanto para mulheres:

In [4]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Dados de exemplo
x1 = df['exp']
y1 = df['salario']
sexo1 = df['sexo'] == 'Feminino'  # cria uma lista de booleanos para indicar o sexo correspondente a cada ponto
cores1 = ['red' if s else 'blue' for s in sexo1]  # atribui a cor vermelha para o sexo feminino e azul para o sexo masculino

x2 = df['posicao']
y2 = df['salario']
sexo2 = df['sexo'] == 'Feminino'  # cria uma lista de booleanos para indicar o sexo correspondente a cada ponto
cores2 = ['red' if s else 'blue' for s in sexo2]  # atribui a cor vermelha para o sexo feminino e azul para o sexo masculino

# Criação dos objetos Scatter
scatter1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    name='Gráfico 1',
    marker=dict(color=cores1)  # atribui as cores de acordo com o sexo
)

scatter2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    name='Gráfico 2',
    marker=dict(color=cores2)  # atribui as cores de acordo com o sexo
)

# Criação dos objetos Scatter
scatter1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    name='Gráfico 1',
    marker=dict(color=cores1)  # atribui as cores de acordo com o sexo
)

scatter2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    name='Gráfico 2',
    marker=dict(color=cores2)  # atribui as cores de acordo com o sexo
)

# Criação do objeto Layout
layout = go.Layout(
    #title='Dois gráficos de dispersão',
    xaxis=dict(title='Salário (dolar)'),  # título do eixo x do primeiro gráfico
    yaxis=dict(title='Experiência (anos)'),  # título do eixo y do primeiro gráfico
    xaxis2=dict(title='Salário (dolar)'),  # título do eixo x do segundo gráfico
    yaxis2=dict(title='Posição na Empresa'),  # título do eixo y do segundo gráfico
)

# Criação do objeto Figure
fig = make_subplots(rows=1, cols=2)
fig.add_trace(scatter1, row=1, col=1)
fig.add_trace(scatter2, row=1, col=2)

fig.update_layout(layout)

fig.show()

Aqui conseguimos ver o crescimento mais claro na posição da empresa por salário e o comportamento menos linear de crescimento no experiencia por salário e com uma dispersão grande.

Baseado no segundo grafico, é possivel imaginar uma regressão do tipo 



$$ yi = α + β1sexoi + β2experi + β3posici + Ei $$

Mas esse tipo de regressão não seria totalmente honesta em descrever o todo pois isso seria baseado em uma analise de duas variaveis deixando de lado as demais. A grande sacado do metodo de Akaike é conseguir entender o comportamento do todo.

O método de Akaike é uma abordagem de seleção de modelo que visa encontrar o modelo que melhor representa os dados, levando em conta tanto a adequação do modelo quanto a complexidade do mesmo. A ideia principal é que um modelo bom deve ser capaz de descrever bem os dados observados, mas não deve ser muito complexo, de modo a evitar o overfitting.

Para selecionar o modelo mais adequado, o método de Akaike utiliza o Critério de Informação de Akaike (AIC), que é uma medida que combina a qualidade de ajuste do modelo aos dados com sua complexidade, expressa pelo número de parâmetros que o modelo possui. O AIC é uma medida de informação relativa, o que significa que o valor do AIC de um modelo só faz sentido em relação ao valor do AIC de outros modelos ajustados aos mesmos dados.

O cálculo do AIC envolve a estimativa da função de verossimilhança do modelo, que é uma medida de quão bem o modelo representa os dados observados. A partir da função de verossimilhança, o AIC é calculado como 


$$ 2k - 2ln(L) $$


onde k é o número de parâmetros do modelo e L é a máxima verossimilhança do modelo.

O objetivo do método de Akaike é encontrar o modelo com o menor valor de AIC, o que indica o melhor compromisso entre a adequação do modelo e sua complexidade. Em outras palavras, o modelo com o menor AIC é aquele que melhor descreve os dados observados com o menor número possível de parâmetros.


In [5]:
df.dtypes

salario      int64
sexo        object
posicao      int64
exp        float64
dtype: object

In [6]:
# mapeia a categoria 'M' para o valor 0 e a categoria 'F' para o valor 1
df['sexo'] = df['sexo'].map({'Masculino': 0, 'Feminino': 1})

print(df)

     salario  sexo  posicao   exp
0        148     0        7  16.7
1        165     0        7   6.7
2        145     0        5  14.8
3        139     1        7  13.9
4        142     1        6   6.4
..       ...   ...      ...   ...
215      146     0        5   8.9
216      147     0        5   8.8
217      156     0        7  15.1
218      132     0        4   4.7
219      161     0        7  16.5

[220 rows x 4 columns]


# Implementando o metodo

No livro, foi feita uma engenharia de feature pra descobri qual seria a melhor combinação das variaveis que daria o menor Akaike para nosso modelo.

In [35]:
import itertools

# Crie uma lista com todas as combinações possíveis de duas variáveis
combinacoes = list(itertools.combinations(['sexo', 'posicao', 'exp'], 2))

# Crie uma variável para armazenar o menor AIC encontrado
melhor_aic = float('inf')

# Loop pelas combinações e ajuste o modelo para cada uma
for comb in combinacoes:
    # Crie uma nova coluna com a combinação das variáveis
    nome_coluna = comb[0] + '_' + comb[1]
    df[nome_coluna] = df[comb[0]] * df[comb[1]]

    # Ajuste o modelo e calcule o AIC
    X = df[['sexo', 'posicao', 'exp', nome_coluna]]
    X = sm.add_constant(X)
    modelo = sm.OLS(y, X)
    resultado = modelo.fit()
    aic = resultado.aic

    # Verifique se essa combinação tem um AIC menor que o atual melhor AIC
    if aic < melhor_aic:
        melhor_aic = aic
        melhor_combinacao = comb
        melhor_nome_coluna = nome_coluna

# Imprima a melhor combinação e o nome da nova coluna
print("Melhor combinação:", melhor_combinacao)
print("Nome da nova coluna:", melhor_nome_coluna)


Melhor combinação: ('posicao', 'exp')
Nome da nova coluna: posicao_exp


Com isso conseguimos os mesmo valores de coeficientes e uma constante proxima no livro a constate deu 108,042

In [38]:
print('Constante:', resultado.params[0])
print('Coeficientes:', resultado.params[1:])

Constante: 105.23116936866123
Coeficientes: sexo           2.810991
posicao        8.095985
exp            0.336507
posicao_exp   -0.134932
dtype: float64


In [45]:
# Aplicando no modelo treinado 
exp = 5
posicao = 4
sexo = 0  # 0 corresponde a homem e 1 a mulher

# Criando um array com os valores das variáveis explicativas para o novo exemplo
novos_dados = [sexo, posicao, exp, posicao*exp, 1] # adicionando a constante da regressão

# Fazendo a previsão do salário para o novo exemplo
salario_previsto = resultado.predict(novos_dados)

print(f"O salário previsto para um executivo homem com 5 anos de experiência e na posição 4 é de ${salario_previsto[0]:.2f}")


O salário previsto para um executivo homem com 5 anos de experiência e na posição 4 é de $58.32


Não sei o que saiu de errado professor!!!